# Train Vision Transformer for image classification with HuggingFace Trainer + the SageMaker Model Parallelism Library

This notebook walks you through how to use HuggingFace Trainer with the SageMaker Model Parallelism library to train VIT(Vision Transformer). You'll learn how to train the VIT model with tensor parallelism on beans dataset.

**Note**: To run this example training job, you must be in `us-east-1` or `us-west-2`. The preview version of container images are available only in those two regions.

## Install and Upgrade Libraries

The SageMaker model parallelism library's tensor parallelism feature requires the SageMaker Python SDK and the SageMaker Experiments library. Run the following cell to install or upgrade the libraries.

**Note:** To finish applying the changes, you must restart the kernel.

In [ ]:
# # run once, restart kernel, then comment out this cell
# # update sagemaker to the latest 2.x version
#! pip3 install -qU pip
#! pip3 install -qU "sagemaker>=2,<3"
#! pip3 install -qU sagemaker-experiments

# import IPython
# IPython.Application.instance().kernel.do_shutdown(True)

Import and check if the SageMaker Python SDK version is successfully set to the latest version

In [ ]:
import sagemaker

print(sagemaker.__version__)

## Amazon SageMaker Initialization

This private preview feature is available to use in `us-east-1` and `us-west-2`.
Throughout this example, you'll use a training script of GPT model and a text dataset.

Run the following cell to import SageMaker modules and retrieve information of your current SageMaker work environment: your AWS account ID, the AWS Region you are using to run the notebook, and the ARN of your Amazon SageMaker execution role.

In [ ]:
%%time
import os

from sagemaker import get_execution_role
from sagemaker.huggingface import HuggingFace
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
import boto3

# If running in Sagemaker notebook this can stay commented
# os.environ["AWS_PROFILE"] = "sm"

# The only supported region is us-west-2
# since the preview image is only in this region

os.environ["AWS_DEFAULT_REGION"] = "us-west-2"

role = (
    get_execution_role()
)  # provide a pre-existing role ARN as an alternative to creating a new role
print(f"SageMaker Execution Role:{role}")

client = boto3.client("sts")
account = client.get_caller_identity()["Account"]
print(f"AWS account:{account}")

session = boto3.session.Session()
region = session.region_name
print(f"AWS region:{region}")

sm_boto_client = boto3.client("sagemaker")
sagemaker_session = sagemaker.session.Session(boto_session=session)

## Specify Amazon S3 Bucket Paths

Here you need to specify the paths for training data to be used by your job. The bucket used must be in the same region as where training will run. As part of the private preview artifacts, we provide a synthetic dataset that you can use to quickly get started in 'smdistributed-modelparallel-preview' bucket. This bucket is in us-west-2, and we recommend you copy the data to your own bucket and update the paths in the next cell to avoid any cross-account permission issues depending on your IAM role permissions.

After you successfully run this example tensor parallel training job, you can modify the S3 bucket to where your own dataset is stored.

In [169]:
# s3_train_bucket = "s3://{train}-{dataset}/"
# s3_test_bucket = "s3://{test}-{dataset}/"

The below bucket will store output artifacts of the training job. You can modify this as needed.

In [170]:
# s3_output_bucket = f"s3://sagemaker-{region}-{account}/smp-tensorparallel-beta/gpt_synthetic_simpletrainer_outputdir/"
s3_output_bucket = f"s3://{output_bucket}]"

## Define Data Channels for SageMaker Training

In this step, you define SageMaker training data channels using the above buckets.  

In [143]:
# train = sagemaker.inputs.TrainingInput(s3_train_bucket, distribution='FullyReplicated', s3_data_type='S3Prefix')
# test = sagemaker.inputs.TrainingInput(s3_test_bucket, distribution='FullyReplicated', s3_data_type='S3Prefix')
# data_channels = {'train': train, 'test': test}

## Setup fsx and use fsx for data channels and checkpoints

While the above option is easier to setup, using an FSX can be beneficial for performance when dealing with large input sizes and large model sizes. If you are using models above 13B, checkpointing should be done using FSX. 

Please see the instructions [here](https://github.com/aws/amazon-sagemaker-examples/blob/master/advanced_functionality/distributed_tensorflow_mask_rcnn/mask-rcnn-scriptmode-fsx.ipynb), to create the FSx lustre filesystem and import the dataset from the S3 bucket to your fsx filesystem. Note that the FSX must be created in a private subnet with internet gateway to ensure that training job has access to the internet. 

In [144]:
# Instructions obtained from:
# https://github.com/aws/amazon-sagemaker-examples/blob/master/advanced_functionality/distributed_tensorflow_mask_rcnn/mask-rcnn-scriptmode-fsx.ipynb

## Set Up Hyperparameters, Metric Definitions, and MPI Options
The following `hyperparameters` dictionary is to pass arguments to the training script (`train_gpt_simple.py`) and set the model parallel configuration when creating the training job.

You can also add custom mpi flags. By default, we have `--mca btl_vader_single_copy_mechanism none` to remove unnecessary logs.

Next we add a base metric definitions to enable the metric upload in SageMaker. You can add any further metric definitions.

In [174]:
save_steps = 50
max_steps = 100

hyperparameters = {
    "output_dir": "/opt/ml/checkpoints",
    "dataset_name": "beans",
    "learning_rate": 0.00002,
    "do_train": "",
    "do_eval": "",
    "save_steps": save_steps,
    "max_steps": max_steps,
    "per_device_train_batch_size": 2,
    "per_device_eval_batch_size": 2,
    "eval_steps": 20,
    "evaluation_strategy": "steps",
    "save_strategy": "steps",
    "save_total_limit": 10,
    "logging_steps": 1,
    "remove_unused_columns": "False",
    "logging_strategy": "steps",
    "seed": 1337,
}

# The checkpoint path (hyperparameters['checkpoint-dir'] or checkpoint_s3_uri) is not unique per job.
# You need to modify as needed for different runs.
# If same path is used for unrelated runs, this may increase time when downloading unnecessary checkpoints,
# and cause conflicts when loading checkpoints.


mpioptions = "-x NCCL_DEBUG=WARN -x SMDEBUG_LOG_LEVEL=ERROR "
mpioptions += (
    "-x SMP_DISABLE_D2D=1 -x SMP_D2D_GPU_BUFFER_SIZE_BYTES=1 -x SMP_NCCL_THROTTLE_LIMIT=1 "
)
mpioptions += "-x FI_EFA_USE_DEVICE_RDMA=1 -x FI_PROVIDER=efa -x RDMAV_FORK_SAFE=1"

metric_definitions = [
    {"Name": "base_metric", "Regex": "<><><><><><>"}
]  # Add your custom metric definitions

## Set Up SageMaker Studio Experiment
Create or load [SageMaker Experiment](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html) for the example training job. This will create an experiment trial object in SageMaker Studio.

In [175]:
from time import gmtime, strftime

# Specify your experiment name
experiment_name = "smp-vit-imae-classification"
# Specify your trial name
trial_name = f"{experiment_name}-trial"

all_experiment_names = [exp.experiment_name for exp in Experiment.list()]
# Load the experiment if it exists, otherwise create
if experiment_name not in all_experiment_names:
    experiment = Experiment.create(
        experiment_name=experiment_name, sagemaker_boto_client=sm_boto_client
    )
else:
    experiment = Experiment.load(
        experiment_name=experiment_name, sagemaker_boto_client=sm_boto_client
    )

# Create the trial
trial = Trial.create(
    trial_name="smp-vit-imae-classification-{}-{}".format(
        trial_name, strftime("%Y-%m-%d-%H-%M-%S", gmtime())
    ),
    experiment_name=experiment.experiment_name,
    sagemaker_boto_client=sm_boto_client,
)

## Specify Essential Parameters for a SageMaker Training Job

Next, you will use the [`SageMaker Estimator API`](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) to define a SageMaker Training Job, passing values through the following parameters for training job name, the number of EC2 instances, the instance type, and the size of the volume attached to the instances. 

* `instance_count`
* `instance_type`
* `volume_size`
* `base_job_name`

### Update the Type and Number of EC2 Instance to Use

The instance type and the number of instances you specify to the `instance_type` and `instance_count` parameters, respectively, will determine the total number of GPUs (world size).

$$ \text{(world size) = (the number of GPUs on a single instance)}\times\text{(the number of instance)}$$

In [176]:
instance_type = "ml.p4d.24xlarge"

instance_count = 1

# set to the number of GPUs on that instance
processes_per_host = 8

To look up the number of GPUs of different instance types, see [Amazon EC2 Instance Types](https://aws.amazon.com/ec2/instance-types/). Use the section **Accelerated Computing** to see general purpose GPU instances. Note that, for example, a given instance type `p4d.24xlarge` has a corresponding instance type `ml.p4d.24xlarge` in SageMaker.
For SageMaker supported `ml` instances and cost information, see [Amazon SageMaker Pricing](https://aws.amazon.com/sagemaker/pricing/). 

### Attach an EBS Volume to the Training Instance
The volume size you specify in `volume_size` must be larger than your input data size. In this example, the volume size is set to 500GB.

In [177]:
volume_size = 500

### Specify a Base Job Name

In [178]:
model_config = "vit"

SM_HP_MP_PARAMETERS = {
    "ddp": True,
    "microbatches": 1,
    "optimize": "speed",
    "pipeline": "interleaved",
    "placement_strategy": "cluster",
    "tensor_parallel_degree": 2,
    "partitions": 1,
    "prescaled_batch": False,
    "shard_optimizer_state": True,
    "fp16": False,
}

machine_str = instance_type.split(".")[1] + instance_type.split(".")[2][:3]
pp_degree = SM_HP_MP_PARAMETERS["partitions"]
tp_degree = SM_HP_MP_PARAMETERS["tensor_parallel_degree"]
base_job_name = f'smp-{model_config}-{machine_str}-tp{tp_degree}-pp{pp_degree}-bs{hyperparameters["per_device_train_batch_size"]}'

### Resume Training from a Previous Checkpoint

In [179]:
resume_from_checkpoint = False
resume_step = 100  # the checkpoint step you want to resume training from

# We label our job with the model configuration and the number of nodes
job_name = f"{model_config}_nodes_{instance_count}"
# Here, we use the same bucket for both checkpoints and outputs
checkpoint_bucket = s3_output_bucket
checkpoint_s3_uri = checkpoint_s3_uri = f"{checkpoint_bucket}/{job_name}/checkpoints"

# The previous checkpoint to load must have the same model config.
if resume_from_checkpoint:
    checkpoint_step = resume_step
    checkpoint_dir = f"/opt/ml/checkpoints/checkpoint-{checkpoint_step}"
    hyperparameters["resume_from_checkpoint"] = checkpoint_dir

### Create a SageMaker PyTorch Estimator

The following cell constructs a PyTorch estimator using the parameters defined above. To see how the SageMaker tensor parallelism modules and functions are applied to the script, see the `train_gpt_simple.py` file and the private preview documentation. 

In [180]:
kwargs = {}

smp_estimator = HuggingFace(
    entry_point="run_vit_image_classification.py",
    source_dir=os.getcwd(),  # copies your current working directory to S3 for SageMaker
    role=role,
    instance_type=instance_type,
    volume_size=volume_size,
    instance_count=instance_count,
    sagemaker_session=sagemaker_session,
    distribution={
        "mpi": {
            "enabled": True,
            "processes_per_host": processes_per_host,
            "custom_mpi_options": mpioptions,
        },
        "smdistributed": {
            "modelparallel": {
                "enabled": True,
                "parameters": {
                    "ddp": True,
                    "tensor_parallel_degree": SM_HP_MP_PARAMETERS["tensor_parallel_degree"],
                    # partitions is a required param in the current SM SDK so it needs to be passed,
                    # these two map to the same config
                    "partitions": SM_HP_MP_PARAMETERS["partitions"],
                    "shard_optimizer_state": SM_HP_MP_PARAMETERS["shard_optimizer_state"],
                    "prescaled_batch": SM_HP_MP_PARAMETERS["prescaled_batch"],
                    "fp16_params": SM_HP_MP_PARAMETERS["fp16"],
                    "optimize": SM_HP_MP_PARAMETERS["optimize"],
                    "auto_partition": True,
                    "default_partition": 0,
                },
            }
        },
    },
    py_version="py38",
    output_path=s3_output_bucket,
    checkpoint_s3_uri=checkpoint_s3_uri,
    checkpoint_local_path=None,
    metric_definitions=metric_definitions,
    hyperparameters=hyperparameters,
    image_uri="763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:1.12.0-gpu-py38-cu113-ubuntu20.04-sagemaker-v1.0",
    debugger_hook_config=False,
    disable_profiler=True,
    base_job_name=base_job_name,
    **kwargs,
)

Finally, run the estimator to launch the SageMaker training job of GPT2 model with tensor parallelism.

In [ ]:
smp_estimator.fit(
    experiment_config={
        "ExperimentName": experiment.experiment_name,
        "TrialName": trial.trial_name,
        "TrialComponentDisplayName": "Training",
    },
    logs=True,
)

# Accessing the Training Logs

You can access the training logs from [Amazon CloudWatch](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/WhatIsCloudWatch.html). Make sure to look at the logs of algo-1 as that is the master node whose output stream will have the training job logs.

You can use CloudWatch to track SageMaker GPU and memory utilization during training and inference. To view the metrics and logs that SageMaker writes to CloudWatch, see *Processing Job, Training Job, Batch Transform Job, and Endpoint Instance Metrics* in [Monitor Amazon SageMaker with Amazon CloudWatch](https://docs.aws.amazon.com/sagemaker/latest/dg/monitoring-cloudwatch.html).

If you are a new user of CloudWatch, see [Getting Started with Amazon CloudWatch](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/GettingStarted.html). 

For additional information on monitoring and analyzing Amazon SageMaker training jobs, see [Monitor and Analyze Training Jobs Using Metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/training-metrics.html).

# Deploying Trained Model for Inference

In most cases the trained model can be deployed on a single device for inference, since inference has smaller memory requirements. You can use the SMP API to create a single, unified model after training. For TensorFlow, a SavedModel can be created using `smp.DistributedModel.save_model` API, and for PyTorch, `smp.save()` can be used.

After you build and train your models, you can deploy them to get predictions in one of two ways:

* To set up a persistent endpoint to get predictions from your models, use SageMaker hosting services. For an overview on deploying a single model or multiple models with SageMaker hosting services, see [Deploy a Model on SageMaker Hosting Services](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-deployment.html#how-it-works-hosting).
* To get predictions for an entire dataset, use SageMaker batch transform. For an overview on deploying a model with SageMaker batch transform, see [Get Inferences for an Entire Dataset with Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html).

To learn more about deploying models for inference using SageMaker, see [Deploy Models for Inference](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html). 
